# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd

df = pd.read_csv('main_task.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
Restaurant_id        40000 non-null object
City                 40000 non-null object
Cuisine Style        30717 non-null object
Ranking              40000 non-null float64
Rating               40000 non-null float64
Price Range          26114 non-null object
Number of Reviews    37457 non-null float64
Reviews              40000 non-null object
URL_TA               40000 non-null object
ID_TA                40000 non-null object
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [2]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [3]:
# create a dict with an average rating by city
ctr = dict()
temp = df.groupby('City').mean()[['Rating']].to_dict('index')
for city, value in temp.items():
    ctr[city] = value['Rating']
print(ctr)

{'Amsterdam': 4.1312154696132595, 'Athens': 4.2300955414012735, 'Barcelona': 3.9612289685442574, 'Berlin': 4.124593967517401, 'Bratislava': 3.9700996677740865, 'Brussels': 3.9089622641509436, 'Budapest': 4.0900735294117645, 'Copenhagen': 3.9855842185128982, 'Dublin': 4.061664190193165, 'Edinburgh': 4.0880872483221475, 'Geneva': 3.9625779625779627, 'Hamburg': 4.021074815595363, 'Helsinki': 3.9468085106382977, 'Krakow': 4.146726862302483, 'Lisbon': 4.046153846153846, 'Ljubljana': 4.112021857923497, 'London': 3.9550981413930866, 'Luxembourg': 3.914285714285714, 'Lyon': 3.9484304932735426, 'Madrid': 3.800836550836551, 'Milan': 3.8248945147679323, 'Munich': 4.047032474804031, 'Oporto': 4.178362573099415, 'Oslo': 3.8935064935064934, 'Paris': 3.9424137226873595, 'Prague': 4.032571032571033, 'Rome': 4.230269489894129, 'Stockholm': 3.870121951219512, 'Vienna': 4.078044596912521, 'Warsaw': 4.089408528198074, 'Zurich': 4.039962825278811}


In [4]:
def replace_price_range(row):
    if row['Price Range'] == '$':
        return 10
    elif row['Price Range'] == '$$ - $$$':
        return 20
    else:
        return 30

In [5]:
def calculate_mean_score(row):
    return ctr.get(row['City'])

In [6]:
df = pd.read_csv('main_task.csv')

# add a default cuisine
cuisine_values = {'Cuisine Style': '[\'Vegetarian Friendly\']'}
df = df.fillna(value=cuisine_values)

df['Cuisine Style'] = df['Cuisine Style'].str.replace('[', '').str.replace(']', '').str.replace('\'', '')
dummies = df['Cuisine Style'].str.get_dummies(sep=', ')
df = df.join(dummies)

# add a default price range instead of null
price_values = {'Price Range': '$$ - $$$'}
df = df.fillna(value=price_values)
df['Prices'] = df.apply(replace_price_range, axis=1)

# add a new column - mean rating per city 
df['CtrCity'] = df.apply(calculate_mean_score, axis=1)

# add a mean value to numeric column instead of null values
review_mean = df['Number of Reviews'].mean(skipna = True) 

review_values = {'Number of Reviews': review_mean}
df = df.fillna(value=review_values)

# drop non numeric columns
df = df.drop(['City', 'Price Range', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 131 entries, Restaurant_id to CtrCity
dtypes: float64(4), int64(126), object(1)
memory usage: 40.0+ MB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [7]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [8]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [9]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [10]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [11]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [12]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
# Hint: The less, the better 
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
# 0.42473367440476195 - default value
# 0.224704 - the best with mean value per city
# 0.21676399999999998 - with dummies

MAE: 0.21676399999999998
